In [1]:
import pandas as pd
import numpy as np
import optuna
import polars as pl

from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin




In [12]:
train_df = pl.read_csv("processsing/train_table_merged.csv")
test_df = pl.read_csv("processsing/test_table_merged.csv")

In [7]:
train_df.head(5)

SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,…,DAYS_LAST_DUE_mean,DAYS_TERMINATION_mean,NFLAG_INSURED_ON_APPROVAL_mean,CNT_INSTALMENT_mean,CNT_INSTALMENT_FUTURE_mean,POS_MONTHS_BALANCE_std,CNT_INSTALMENT_std,CNT_INSTALMENT_FUTURE_std,NUM_INSTALMENT_NUMBER_mean,DAYS_INSTALMENT_mean,DAYS_ENTRY_PAYMENT_mean,AMT_INSTALMENT_mean,AMT_PAYMENT_mean,NUM_INSTALMENT_NUMBER_std,DAYS_INSTALMENT_std,DAYS_ENTRY_PAYMENT_std,AMT_INSTALMENT_std,MONTHS_BALANCE_mean,AMT_BALANCE_mean,AMT_CREDIT_LIMIT_ACTUAL_mean,AMT_DRAWINGS_ATM_CURRENT_mean,AMT_DRAWINGS_CURRENT_mean,AMT_DRAWINGS_OTHER_CURRENT_mean,AMT_DRAWINGS_POS_CURRENT_mean,AMT_INST_MIN_REGULARITY_mean,AMT_PAYMENT_CURRENT_mean,AMT_PAYMENT_TOTAL_CURRENT_mean,AMT_RECEIVABLE_PRINCIPAL_mean,AMT_RECIVABLE_mean,AMT_TOTAL_RECEIVABLE_mean,CNT_DRAWINGS_ATM_CURRENT_mean,CNT_DRAWINGS_CURRENT_mean,CNT_DRAWINGS_OTHER_CURRENT_mean,CNT_DRAWINGS_POS_CURRENT_mean,CNT_INSTALMENT_MATURE_CUM_mean,SK_DPD_mean,SK_DPD_DEF_mean
i64,i64,str,str,str,str,i64,f64,f64,f64,f64,str,str,str,str,str,f64,i64,i64,f64,i64,f64,i64,i64,i64,i64,i64,i64,str,f64,i64,i64,str,i64,i64,i64,i64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
100002,1,"""Cash loans""","""M""","""N""","""Y""",0,202500.0,406597.5,24700.5,351000.0,"""Unaccompanied""","""Working""","""Secondary / se…","""Single / not m…","""House / apartm…",0.018801,-9461,-637,-3648.0,-2120,null,1,1,0,1,1,0,"""Laborers""",1.0,2,2,"""WEDNESDAY""",10,0,0,0,…,-25.0,-17.0,0.0,24.0,15.0,5.627314,0.0,5.627314,10.0,-295.0,-315.421053,11559.247105,11559.247105,5.627314,168.81943,172.058877,10058.037722,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
100003,0,"""Cash loans""","""F""","""N""","""N""",0,270000.0,1293502.5,35698.5,1.1295e6,"""Family""","""State servant""","""Higher educati…","""Married""","""House / apartm…",0.003541,-16765,-1188,-1186.0,-291,null,1,1,0,1,1,0,"""Core staff""",2.0,1,1,"""MONDAY""",11,0,0,0,…,-1054.333333,-1047.333333,0.666667,9.791667,5.666667,2.834844,0.589256,3.209241,4.666667,-1164.333333,-1171.781746,78558.479286,78558.479286,2.545542,76.366269,76.380137,58313.693509,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
100004,0,"""Revolving loan…","""M""","""Y""","""Y""",0,67500.0,135000.0,6750.0,135000.0,"""Unaccompanied""","""Working""","""Secondary / se…","""Single / not m…","""House / apartm…",0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,"""Laborers""",1.0,2,2,"""MONDAY""",9,0,0,0,…,-724.0,-714.0,0.0,3.75,2.25,1.290994,0.5,1.707825,2.0,-754.0,-761.666667,7096.155,7096.155,1.0,30.0,34.019602,3011.87181,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
100006,0,"""Cash loans""","""F""","""N""","""Y""",0,135000.0,312682.5,29686.5,297000.0,"""Unaccompanied""","""Working""","""Secondary / se…","""Civil marriage…","""House / apartm…",0.008019,-19005,-3039,-9833.0,-2437,null,1,1,0,1,0,0,"""Laborers""",2.0,2,2,"""WEDNESDAY""",17,0,0,0,…,182477.5,182481.75,0.0,12.888889,10.214286,2.159047,10.183704,11.929286,3.166667,-260.666667,-285.966667,241944.225,241944.225,2.304395,69.131838,74.169251,2834.601883,-3.5,0.0,270000.0,null,0.0,null,null,0.0,null,0.0,0.0,0.0,0.0,null,0.0,null,null,0.0,0.0,0.0
100007,0,"""Cash loans""","""M""","""

In [13]:
test_df.head(5)

SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,…,DAYS_LAST_DUE_mean,DAYS_TERMINATION_mean,NFLAG_INSURED_ON_APPROVAL_mean,CNT_INSTALMENT_mean,CNT_INSTALMENT_FUTURE_mean,POS_MONTHS_BALANCE_std,CNT_INSTALMENT_std,CNT_INSTALMENT_FUTURE_std,NUM_INSTALMENT_NUMBER_mean,DAYS_INSTALMENT_mean,DAYS_ENTRY_PAYMENT_mean,AMT_INSTALMENT_mean,AMT_PAYMENT_mean,NUM_INSTALMENT_NUMBER_std,DAYS_INSTALMENT_std,DAYS_ENTRY_PAYMENT_std,AMT_INSTALMENT_std,MONTHS_BALANCE_mean,AMT_BALANCE_mean,AMT_CREDIT_LIMIT_ACTUAL_mean,AMT_DRAWINGS_ATM_CURRENT_mean,AMT_DRAWINGS_CURRENT_mean,AMT_DRAWINGS_OTHER_CURRENT_mean,AMT_DRAWINGS_POS_CURRENT_mean,AMT_INST_MIN_REGULARITY_mean,AMT_PAYMENT_CURRENT_mean,AMT_PAYMENT_TOTAL_CURRENT_mean,AMT_RECEIVABLE_PRINCIPAL_mean,AMT_RECIVABLE_mean,AMT_TOTAL_RECEIVABLE_mean,CNT_DRAWINGS_ATM_CURRENT_mean,CNT_DRAWINGS_CURRENT_mean,CNT_DRAWINGS_OTHER_CURRENT_mean,CNT_DRAWINGS_POS_CURRENT_mean,CNT_INSTALMENT_MATURE_CUM_mean,SK_DPD_mean,SK_DPD_DEF_mean
i64,str,str,str,str,i64,f64,f64,f64,f64,str,str,str,str,str,f64,i64,i64,f64,i64,f64,i64,i64,i64,i64,i64,i64,str,f64,i64,i64,str,i64,i64,i64,i64,i64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
100001,"""Cash loans""","""F""","""N""","""Y""",0,135000.0,568800.0,20560.5,450000.0,"""Unaccompanied""","""Working""","""Higher educati…","""Married""","""House / apartm…",0.01885,-19241,-2329,-5170.0,-812,null,1,1,0,1,0,1,null,2.0,2,2,"""TUESDAY""",18,0,0,0,0,…,-1619.0,-1612.0,0.0,4.0,2.0,1.581139,0.0,1.581139,2.5,-1664.0,-1679.5,7312.725,7312.725,1.290994,38.729833,43.02325,6723.45,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
100005,"""Cash loans""","""M""","""N""","""Y""",0,99000.0,222768.0,17370.0,180000.0,"""Unaccompanied""","""Working""","""Secondary / se…","""Married""","""House / apartm…",0.035792,-18064,-4469,-9118.0,-1623,null,1,1,0,1,0,0,"""Low-skill Labo…",2.0,2,2,"""FRIDAY""",9,0,0,0,0,…,-466.0,-460.0,0.0,11.7,7.2,3.316625,0.948683,3.614784,5.0,-586.0,-609.555556,6240.205,6240.205,2.738613,82.158384,90.554005,4281.015,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
100013,"""Cash loans""","""M""","""Y""","""Y""",0,202500.0,663264.0,69777.0,630000.0,null,"""Working""","""Higher educati…","""Married""","""House / apartm…",0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,"""Drivers""",2.0,2,2,"""MONDAY""",14,0,0,0,0,…,-715.666667,-710.333333,0.333333,16.648148,11.451178,3.605137,1.492781,4.518695,6.027778,-854.833333,-867.592593,16349.077917,13702.794792,3.127653,93.8296,97.056815,23427.198086,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
100028,"""Cash loans""","""F""","""N""","""Y""",2,315000.0,1.575e6,49018.5,1.575e6,"""Unaccompanied""","""Working""","""Secondary / se…","""Married""","""House / apartm…",0.026392,-13976,-1866,-2000.0,-4208,null,1,1,0,1,1,0,"""Sales staff""",4.0,2,2,"""WEDNESDAY""",11,0,0,0,0,…,121171.333333,121182.666667,0.0,15.928571,8.3125,4.615657,1.336306,5.87108,17.595238,-944.964286,-949.814286,7836.897982,7557.738339,9.916709,202.263805,203.25224,6201.859346,-25.0,8085.058163,225000.0,613.636364,6156.400408,0.0,6242.355,6133.363929,6598.243256,5606.152347,7680.352041,7968.609184,7968.609184,0.045455,2.387755,0.0,2.

##  Preprocessing steps

### Dropping features with too many NaNs

In [8]:
COLS_TO_DROP_HIGH_NAN = ['OWN_CAR_AGE', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'FLOORSMIN_AVG',
       'LIVINGAPARTMENTS_AVG', 'NONLIVINGAPARTMENTS_AVG', 'YEARS_BUILD_MODE',
       'COMMONAREA_MODE', 'FLOORSMIN_MODE', 'LIVINGAPARTMENTS_MODE',
       'NONLIVINGAPARTMENTS_MODE', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI',
       'FLOORSMIN_MEDI', 'LIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_MEDI',
       'FONDKAPREMONT_MODE', 'meanMONTHS_BALANCE_mean',
       'STD_MONTHS_BALANCE_mean', 'lastSTATUS_mode', 'modeSTATUS_mode',
       'RATE_INTEREST_PRIMARY_mean', 'RATE_INTEREST_PRIVILEGED_mean',
       'MONTHS_BALANCE_mean', 'AMT_BALANCE_mean',
       'AMT_CREDIT_LIMIT_ACTUAL_mean', 'AMT_DRAWINGS_ATM_CURRENT_mean',
       'AMT_DRAWINGS_CURRENT_mean', 'AMT_DRAWINGS_OTHER_CURRENT_mean',
       'AMT_DRAWINGS_POS_CURRENT_mean', 'AMT_INST_MIN_REGULARITY_mean',
       'AMT_PAYMENT_CURRENT_mean', 'AMT_PAYMENT_TOTAL_CURRENT_mean',
       'AMT_RECEIVABLE_PRINCIPAL_mean', 'AMT_RECIVABLE_mean',
       'AMT_TOTAL_RECEIVABLE_mean', 'CNT_DRAWINGS_ATM_CURRENT_mean',
       'CNT_DRAWINGS_CURRENT_mean', 'CNT_DRAWINGS_OTHER_CURRENT_mean',
       'CNT_DRAWINGS_POS_CURRENT_mean', 'CNT_INSTALMENT_MATURE_CUM_mean',
       'SK_DPD_mean', 'SK_DPD_DEF_mean']

In [14]:
train_df = train_df.drop(COLS_TO_DROP_HIGH_NAN)
test_df = test_df.drop(COLS_TO_DROP_HIGH_NAN)

### Imputing NaN-vals


In [73]:
def imputer(df, cat_cols, num_cols):
    """
    Impute missing values in a Polars DataFrame:
    - Numeric columns: replace NaNs with the median.
    - Categorical columns: replace NaNs with 'MISSING'.

    Parameters:
    df (pl.DataFrame): The Polars DataFrame to be imputed.

    Returns:
    pl.DataFrame: The imputed DataFrame.
    """
    for col in num_cols:
        median = df[col].median()
        df = df.with_columns([df[col].fill_null(median)])

    # Impute categorical columns with 'MISSING'
    for col in cat_cols:
        df = df.with_columns([df[col].fill_null("MISSING")])

    return df
    
    
    


In [79]:
#train_df.columns
numeric_cols = test_df.select(pl.col(pl.datatypes.NUMERIC_DTYPES)).columns
categorical_cols = test_df.select(pl.col(pl.datatypes.Utf8)).columns


In [80]:
#Imputing 

train_df_impt = imputer(train_df, categorical_cols, numeric_cols)
test_df_impt = imputer(test_df, categorical_cols, numeric_cols)